In [1]:
import json
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from IPython.core.debugger import set_trace


from chaitjo_graph_convnet_tsp import config as chaitjo_config
from chaitjo_graph_convnet_tsp.models.gcn_model import ResidualGatedGCNModel
from chaitjo_graph_convnet_tsp.utils.graph_reader import GraphReader


In [2]:
model_path = 'chaitjo_graph_convnet_tsp/'
graph_path = 'dataset/round6/node/'
feat_path = 'dataset/round6/feat300/'

In [3]:
config_path = model_path + 'tsp-models/tsp20/config.json'
config = chaitjo_config.Settings(json.load(open(config_path)))
print(config)

{'expt_name': 'tsp20', 'gpu_id': '1', 'train_filepath': './data/tsp20_train_concorde.txt', 'val_filepath': './data/tsp20_val_concorde.txt', 'test_filepath': './data/tsp20_test_concorde.txt', 'num_nodes': 20, 'num_neighbors': -1, 'node_dim': 2, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 300, 'num_layers': 30, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}


In [4]:
dtypeFloat = torch.cuda.FloatTensor
dtypeLong = torch.cuda.LongTensor

net = nn.DataParallel(ResidualGatedGCNModel(config, dtypeFloat, dtypeLong))
print(net)

DataParallel(
  (module): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=300, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=150, bias=False)
    (edges_embedding): Embedding(3, 150)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (edge_feat): EdgeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (bn_node): BatchNormNode(
          (batch_norm): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
        (bn_edge): BatchNormEdge(
          (batch_norm): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
      )
      (1): Re

In [6]:
checkpoint_path = model_path + 'tsp-models/tsp20/best_val_checkpoint.tar'
checkpoint = torch.load(checkpoint_path)
net.load_state_dict(checkpoint['model_state_dict'])
net.cuda()
net.eval()

DataParallel(
  (module): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=300, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=150, bias=False)
    (edges_embedding): Embedding(3, 150)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (edge_feat): EdgeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (bn_node): BatchNormNode(
          (batch_norm): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
        (bn_edge): BatchNormEdge(
          (batch_norm): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
      )
      (1): Re

In [7]:
for batch in GraphReader(graph_path):
    print(batch)
    break

{'edges': array([[[2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1

In [9]:
with torch.no_grad():
    for batch in GraphReader(graph_path):
        
        # Convert batch to torch Variables
        x_edges = Variable(torch.LongTensor(batch.edges).type(dtypeLong), requires_grad=False)
        x_edges_values = Variable(torch.FloatTensor(batch.edges_values).type(dtypeFloat), requires_grad=False)
        x_nodes = Variable(torch.LongTensor(batch.nodes).type(dtypeLong), requires_grad=False)
        x_nodes_coord = Variable(torch.FloatTensor(batch.nodes_coord).type(dtypeFloat), requires_grad=False)
        gFilename = batch.gFilename
#         set_trace()
        # Forward pass
        feats = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord)
        feats = np.squeeze(feats.detach().cpu().numpy(), axis=0)
        print(gFilename)
        np.save(feat_path + gFilename, feats)



oCEoHR7_24
pVkXeMn_28
qmAUDZZ_21
aziBY0G_29
qlrCtHq_29
soTgJJO_28
x8j9XsP_26
ZNzt7GM_27
MQuA9b5_23
n1Dzk1t_22
r9g59LH_28
rq47Q4D_28
CG0g9vI_25
FUd4K4c_26
QBy2Vcx_24
7SmSobt_27
LjvKYNO_28
eCkZDJO_25
YiXq5DY_28
Z39Juwz_24
GhJKwVW_28
lumdq3B_26
YJSq80C_27
y1UqNA0_27
TvuhxZp_21
nuMtebk_21
CAs5dl6_25
TCal5gA_29
CnGT8ep_22
HXzM53B_21
93MLHEz_26
cMS3uRH_23
NkZmW5b_27
Cc9jeyh_20
U45E4A9_26
SMgGHXA_26
oZK00F3_24
ml0inkw_22
2h8kZaY_25
rwsV2hN_24
k6FUaS8_28
KETUS95_28
s6ilP6x_25
UsCIA4d_29
qH839cg_23
RaTFgBV_20
39kjV5D_28
3QSsMkV_23
9AmTKfy_22
GAnA9js_20
MMwL1vg_28
sqCwEbm_24
fbs3MjS_21
tZYziJz_22
rqwmXeM_29
ptBzKTi_28
ZxwpOnp_25
fS211jk_21
9J1g1yW_28
h5gxJHe_29
uYw7uMB_20
HF5DWg9_29
1VSD86r_24
kpIZxWd_28
z6YNBm4_25
c754oyY_28
yDCLT94_29
KkIY3ux_27
MK7oz5K_28
yx6dUtJ_27
t7gis1z_21
lOhIXFl_22
8QvdDgH_26
AY7Xwib_25
hwUPn0G_27
YIcl0PY_28
RiHtOkq_21
k5uz0VA_25
Dk7X3Ta_22
ABDU4E2_28
QqY3aNa_25
Rhq3hue_21
UBDZ5jd_29
fsbYJKl_24
END0g4s_22
ym66pUj_26
inJtsto_26
kCkrT0g_23
luVrbmD_29
NU4UjCI_27
oCpqWSQ_27